# SQL in Pandas with SQLAlchemy

We will use the "sshtunnel" library to connect to our remote AWS instance and then pull some data into Pandas using SQLAlchemy and Pyscopg2.

## Creating the SSH Tunnel

The "sshtunnel" library can read an SSH config file, so creating a tunnel is quite easy assuming SSH keys are setup and the SSH config entry has been created. With this setup, the "sshtunnel" library automatically determines what the address of the local port should be.

In [6]:
import pandas as pd
import os

In [23]:
pd.set_option('max_rows', 10)

In [7]:
%load_ext dotenv
%dotenv

%load_ext autoreload
%autoreload 2

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from sshtunnel import SSHTunnelForwarder

AWS_IP_ADDRESS = '54.185.35.146'
AWS_USERNAME = os.environ.get('AWS_USERNAME')
SSH_KEY_PATH = '/Users/brenner/.ssh/id_rsa'

server = SSHTunnelForwarder(
    AWS_IP_ADDRESS,
    ssh_username=AWS_USERNAME,
    ssh_pkey=SSH_KEY_PATH,
    remote_bind_address=('localhost', 5432),
)

server.start()
print(server.is_active, server.is_alive, server.local_bind_port)

True True 53390


In [20]:
pd.set_option('max_columns', None)

##  Connecting via Python
We'll be using a Psycopg2 connector alongside SQLAlchemy to connect to this database.

* **SQLAlchemy:** generates SQL statements
* **Psycopg2:** sends the SQL statements to the Postgres database

    Let's make the connection to the database. Note that the IP address of the Postgres database is 'localhost' and the port is set to whatever the `server` connection above contains. This is because we have used the SSH tunnel to create a connection between the AWS instance and our computer. SSH tunnels enable remote instances to behave as if they are *local*.

In [4]:
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_IP_ADDRESS = 'localhost' ## This is localhost because SSH tunnel is active
POSTGRES_PORT = str(server.local_bind_port)
POSTGRES_USERNAME = os.environ.get('POSTGRES_USERNAME')     ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = os.environ.get('POSTGRES_PASSWORD') ## CHANGE THIS TO YOUR POSTGRES PASSWORD
POSTGRES_DBNAME = 'baseball'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_IP_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)

## Load Some Data!

Pandas has a `read_sql_query` method that will pass a SQL statement to a database connection. Here is an example from the all-star table.

In [9]:
import pandas as pd

pd.read_sql_query('''SELECT * FROM allstarfull LIMIT 5;''', cnx)

,playerid,yearid,gamenum,gameid,teamid,lgid,gp,startingpos
0,gomezle01,1933,0,ALS193307060,NYA,AL,1,1
1,ferreri01,1933,0,ALS193307060,BOS,AL,1,2
2,gehrilo01,1933,0,ALS193307060,NYA,AL,1,3
3,gehrich01,1933,0,ALS193307060,DET,AL,1,4
4,dykesji01,1933,0,ALS193307060,CHA,AL,1,5


And another from the schools table.

In [4]:
pd.read_sql_query('''SELECT * FROM schools LIMIT 5;''', cnx)

,schoolid,schoolname,schoolcity,schoolstate,schoolnick
0,abilchrist,Abilene Christian University,Abilene,TX,Wildcats
1,adelphi,Adelphi University,Garden City,NY,Panthers
2,adrianmi,Adrian College,Adrian,MI,Bulldogs
3,airforce,United States Air Force Academy,Colorado Springs,CO,Falcons
4,akron,University of Akron,Akron,OH,Zips


More sophisticated queries can also be used. This example finds the states with the most schools.

In [5]:
sql_query = '''SELECT schoolstate as state, Count(schoolid) as ct 
               FROM schools 
               GROUP BY state 
               ORDER BY ct DESC 
               LIMIT 5;'''

pd.read_sql_query(sql_query, cnx)

,state,ct
0,PA,57
1,CA,48
2,NY,45
3,TX,41
4,OH,33


Finally, this example finds five players from the year 1985 whose salary was above $500,000.

In [6]:
sql_query = '''SELECT playerid, salary 
               FROM Salaries 
               WHERE yearid = '1985' AND salary > '500000'
               ORDER BY salary DESC
               LIMIT 5;'''

pd.read_sql_query(sql_query, cnx)

,playerid,salary
0,schmimi01,2130300.0
1,cartega01,2028571.0
2,fostege01,1942857.0
3,winfida01,1795704.0
4,gossari01,1713333.0


## Close Server Connection

Finally, we should close the server connection when complete.

In [7]:
server.close()

1. What was the total spent on salaries by each team, each year?

In [13]:
pd.read_sql_query('''
SELECT teamid, sum(salary) as sum
FROM Salaries
GROUP BY teamid''',
                 cnx)

,teamid,sum
0,ML4,2.336458e+08
1,NYA,3.085575e+09
2,PIT,9.173692e+08
3,BOS,2.293681e+09
4,TEX,1.557835e+09
5,HOU,1.372977e+09
6,LAN,2.020752e+09
7,CAL,2.719789e+08
8,COL,1.198507e+09
9,SEA,1.580612e+09


2. What is the first and last year played for each player? *Hint:* Create a new table from 'Fielding.csv'.

In [38]:
pd.read_sql_query('''
SELECT *
FROM Fielding
''',
                 cnx)

,playerid,yearid,stint,teamid,lgid,pos,g,gs,innouts,po,a,e,dp,pb,wp,sb,cs,zr
0,ansonca01,1871,1,RC1,NA,1B,1,NaN,NaN,7.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,biermch01,1871,1,FW1,NA,1B,1,NaN,NaN,9.0,0.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN
2,carleji01,1871,1,CL1,NA,1B,29,NaN,NaN,295.0,4.0,34.0,10.0,NaN,NaN,NaN,NaN,NaN
3,connone01,1871,1,TRO,NA,1B,4,NaN,NaN,35.0,1.0,5.0,1.0,NaN,NaN,NaN,NaN,NaN
4,cravebi01,1871,1,TRO,NA,1B,2,NaN,NaN,18.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
5,fishech01,1871,1,RC1,NA,1B,2,NaN,NaN,12.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN
6,fislewe01,1871,1,PH1,NA,1B,26,NaN,NaN,240.0,1.0,7.0,9.0,NaN,NaN,NaN,NaN,NaN
7,flynncl01,1871,1,TRO,NA,1B,19,NaN,NaN,183.0,8.0,9.0,12.0,NaN,NaN,NaN,NaN,NaN
8,foranji01,1871,1,FW1,NA,1B,16,NaN,NaN,111.0,4.0,16.0,2.0,NaN,NaN,NaN,NaN,NaN
9,fulmech01,1871,1,RC1,NA,1B,1,NaN,NaN,9.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


3. Who has played the most all star games?

In [26]:
pd.read_sql_query('''
SELECT playerid, Count(yearid) as cnt
FROM allstarfull
GROUP BY playerid
ORDER BY cnt DESC
''',
                 cnx)

,playerid,cnt
0,aaronha01,25
1,musiast01,24
2,mayswi01,24
3,mantlmi01,20
4,willite01,19
5,ripkeca01,19
6,carewro01,18
7,yastrca01,18
8,kalinal01,18
9,robinbr01,18


4. Which school has generated the most distinct players? *Hint:* Create new table from 'SchoolsPlayers.csv'.

In [ ]:
query = '''

pd.read_sql_query(query, cnx)

In [10]:
query = '''
CREATE TEMP TABLE temp_table
ON COMMIT DROP
AS
SELECT *
FROM schoolsplayers;

COPY temp_table FROM '/home/brenner/baseballdata/SchoolsPlayers.csv' DELIMITER ',' CSV HEADER;

INSERT INTO schoolsplayers
SELECT DISTINCT playerID
FROM temp_table
'''

pd.read_sql_query(query, cnx)

IntegrityError: (psycopg2.IntegrityError) duplicate key value violates unique constraint "schoolsplayers_pkey"
DETAIL:  Key (playerid)=(fritzla01) already exists.
 [SQL: "\nCREATE TEMP TABLE temp_table\nON COMMIT DROP\nAS\nSELECT *\nFROM schoolsplayers;\n\nCOPY temp_table FROM '/home/brenner/baseballdata/SchoolsPlayers.csv' DELIMITER ',' CSV HEADER;\n\nINSERT INTO schoolsplayers\nSELECT DISTINCT playerID\nFROM temp_table\n"] (Background on this error at: http://sqlalche.me/e/gkpj)

In [69]:
pd.read_sql_query('''
SELECT *
FROM
''', cnx)

,playerid,schoolid,yearmin,yearmax


In [66]:
pd.read_sql_query('''
SELECT *
FROM allstarfull
''', cnx)

,playerid,yearid,gamenum,gameid,teamid,lgid,gp,startingpos
0,gomezle01,1933,0,ALS193307060,NYA,AL,1,1
1,ferreri01,1933,0,ALS193307060,BOS,AL,1,2
2,gehrilo01,1933,0,ALS193307060,NYA,AL,1,3
3,gehrich01,1933,0,ALS193307060,DET,AL,1,4
4,dykesji01,1933,0,ALS193307060,CHA,AL,1,5
...,...,...,...,...,...,...,...,...
4907,alvarpe01,2013,0,NLS201307160,PIT,NL,1,None
4908,corbipa01,2013,0,NLS201307160,ARI,NL,1,None
4909,goldspa01,2013,0,NLS201307160,ARI,NL,1,None
4910,romose01,2013,0,NLS201307160,SFN,NL,0,None


5. Which players have the longest career? Assume that the `debut` and `finalGame` columns comprise the start and end, respectively, of a player's career. *Hint:* Create a new table from 'Master.csv'. Also note that strings can be converted to dates using the [`DATE`](https://wiki.postgresql.org/wiki/Working_with_Dates_and_Times_in_PostgreSQL#WORKING_with_DATETIME.2C_DATE.2C_and_INTERVAL_VALUES) function and can then be subtracted from each other yielding their difference in days.

In [67]:
query = '''
SELECT *
FROM schoolsplayers
'''

pd.read_sql_query(query, cnx)

,playerid,schoolid,yearmin,yearmax


In [68]:
query = '''
SELECT *
FROM Fielding
JOIN allstarfull ON fielding.playerid = allstarfull.playerid
'''

pd.read_sql_query(query, cnx)

,playerid,yearid,stint,teamid,lgid,pos,g,gs,innouts,po,a,e,dp,pb,wp,sb,cs,zr,playerid,yearid,gamenum,gameid,teamid,lgid,gp,startingpos
0,dyeje01,2003,1,OAK,AL,DH,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dyeje01,2006,0,NLS200607110,CHA,AL,1,None
1,dyeje01,2003,1,OAK,AL,DH,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dyeje01,2000,0,NLS200007110,KCA,AL,1,9
2,pinielo01,1974,1,NYA,AL,DH,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pinielo01,1972,0,NLS197207250,KCA,AL,1,None
3,pinsova01,1974,1,KCA,AL,DH,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pinsova01,1960,2,ALS196007130,CIN,NL,1,None
4,pinsova01,1974,1,KCA,AL,DH,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pinsova01,1960,1,ALS196007110,CIN,NL,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136778,youngmi02,2013,1,LAN,NL,SS,1,1.0,21.0,0.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,youngmi02,2006,0,NLS200607110,TEX,AL,1,None
136779,youngmi02,2013,1,LAN,NL,SS,1,1.0,21.0,0.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,youngmi02,2005,0,ALS200507120,TEX,AL,1,None
136780,youngmi02,2013,1,LAN,NL,SS,1,1.0,21.0,0.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,youngmi02,2004,0,NLS200407130,TEX,AL,1,None
136781,zobribe01,2013,1,TBA,AL,SS,21,11.0,345.0,11.0,37.0,1.0,5.0,NaN,NaN,NaN,NaN,NaN,zobribe01,2013,0,NLS201307160,TBA,AL,0,None


In [ ]:
query = '''
SELECT *
FROM Fielding
JOIN allstarfull ON fielding.playerid = allstarfull.playerid
'''

pd.read_sql_query(query, cnx)

In [14]:
query = '''
CREATE TABLE IF NOT EXISTS Master (
    playerID varchar(20) NOT NULL,
    birthYear int DEFAULT NULL,
    birthMonth int DEFAULT NULL,
    birthDay int DEFAULT NULL,
    birthCountry text DEFAULT NULL,
    birthState text DEFAULT NULL,
    birthCity text DEFAULT NULL,
    deathYear int DEFAULT NULL,
    deathMonth int DEFAULT NULL,
    deathDay int DEFAULT NULL,
    deathCountry text DEFAULT NULL,
    deathState text DEFAULT NULL,
    deathCity text DEFAULT NULL,
    nameFirst text DEFAULT NULL,
    nameLast text DEFAULT NULL,
    nameGiven text DEFAULT NULL,
    weight int DEFAULT NULL,
    height int DEFAULT NULL,
    bats text DEFAULT NULL,
    throws text DEFAULT NULL,
    debut text DEFAULT NULL,
    finalGame text DEFAULT NULL,
    retroID varchar(20) DEFAULT NULL,
    bbrefID varchar(20) DEFAULT NULL,
    PRIMARY KEY (nameLast, nameFirst));

COPY Master FROM '/home/brenner/baseballdata/Master.csv' DELIMITER ',' CSV HEADER;
'''

pd.read_sql_query(query, cnx)

IntegrityError: (psycopg2.IntegrityError) duplicate key value violates unique constraint "master_pkey"
DETAIL:  Key (playerid)=(aardsda01) already exists.
CONTEXT:  COPY master, line 2
 [SQL: "\n\nCREATE TABLE IF NOT EXISTS Master (\n    playerID varchar(20) NOT NULL,\n    birthYear int DEFAULT NULL,\n    birthMonth int DEFAULT NULL,\n    birthDay int DEFAULT NULL,\n    birthCountry text DEFAULT NULL,\n    birthState text DEFAULT NULL,\n    birthCity text DEFAULT NULL,\n    deathYear int DEFAULT NULL,\n    deathMonth int DEFAULT NULL,\n    deathDay int DEFAULT NULL,\n    deathCountry text DEFAULT NULL,\n    deathState text DEFAULT NULL,\n    deathCity text DEFAULT NULL,\n    nameFirst text DEFAULT NULL,\n    nameLast text DEFAULT NULL,\n    nameGiven text DEFAULT NULL,\n    weight int DEFAULT NULL,\n    height int DEFAULT NULL,\n    bats text DEFAULT NULL,\n    throws text DEFAULT NULL,\n    debut text DEFAULT NULL,\n    finalGame text DEFAULT NULL,\n    retroID varchar(20) DEFAULT NUL

In [24]:
query = '''
SELECT *
FROM Master
'''

pd.read_sql_query(query, cnx)

,playerid,birthyear,birthmonth,birthday,birthcountry,birthstate,birthcity,deathyear,deathmonth,deathday,deathcountry,deathstate,deathcity,namefirst,namelast,namegiven,weight,height,bats,throws,debut,finalgame,retroid,bbrefid
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,None,None,None,David,Aardsma,David Allan,205.0,75.0,R,R,2004-04-06,2013-09-28,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,None,None,None,Hank,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,USA,GA,Atlanta,Tommie,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,None,None,None,Don,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,None,None,None,Andy,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18349,zupcibo01,1966.0,8.0,18.0,USA,PA,Pittsburgh,NaN,NaN,NaN,None,None,None,Bob,Zupcic,Robert,220.0,76.0,R,R,1991-09-07,1994-08-04,zupcb001,zupcibo01
18350,zupofr01,1939.0,8.0,29.0,USA,CA,San Francisco,2005.0,3.0,25.0,USA,CA,Burlingame,Frank,Zupo,Frank Joseph,182.0,71.0,L,R,1957-07-01,1961-05-09,zupof101,zupofr01
18351,zuvelpa01,1958.0,10.0,31.0,USA,CA,San Mateo,NaN,NaN,NaN,None,None,None,Paul,Zuvella,Paul,173.0,72.0,R,R,1982-09-04,1991-05-02,zuvep001,zuvelpa01
18352,zuverge01,1924.0,8.0,20.0,USA,MI,Holland,NaN,NaN,NaN,None,None,None,George,Zuverink,George,195.0,76.0,R,R,1951-04-21,1959-06-15,zuveg101,zuverge01


In [64]:
query = '''
SELECT nameFirst, nameLast, DATE(finalgame) - DATE(debut) AS day_diff
FROM Master
WHERE DATE(finalgame) > DATE(debut)
ORDER BY day_diff DESC
'''

pd.read_sql_query(query, cnx)

,namefirst,namelast,day_diff
0,Nick,Altrock,12862
1,Jim,O'Rourke,11836
2,Minnie,Minoso,11492
3,Charley,O'Leary,11126
4,Arlie,Latham,10678
...,...,...,...
17155,Bill,Zies,1
17156,John,Smith,1
17157,Ed,Atkinson,1
17158,Wally,Kopf,1


2018-10-23 12:27:32,893| ERROR   | Socket exception: Connection reset by peer (54)
